In [4]:
from sklearn.datasets import make_blobs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn import metrics
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [6]:
data = pd.read_csv("data_cleaned.csv")
data

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q91,Q92,Q93,Q94,Q95,Q96,Q97,Q98,Q99,DLEAVING
0,4,3,5,4,4,4,5,4,4,4,...,3,2,1,3,1,4,4,4,4,1
1,5,5,5,5,5,5,5,4,4,5,...,3,2,1,4,1,4,4,4,4,1
2,5,5,5,5,5,5,5,5,5,5,...,2,1,1,5,2,5,5,5,5,0
3,2,2,5,4,3,2,4,4,4,4,...,3,2,1,1,2,3,1,3,4,0
4,5,5,4,5,5,5,5,5,5,5,...,1,2,1,4,2,5,5,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111128,4,4,5,5,2,5,5,4,5,4,...,4,2,1,4,1,5,5,5,5,1
111129,4,4,4,4,2,3,4,4,4,3,...,3,2,2,4,1,4,4,4,4,1
111130,3,3,3,4,3,4,4,4,4,3,...,2,1,1,3,2,3,3,3,3,1
111131,5,4,5,5,4,5,5,5,4,4,...,2,1,1,5,1,5,4,5,5,1


In [10]:
from sklearn.model_selection import train_test_split
y=data['DLEAVING']
X=data.iloc[:, :-1]
X_train, X_test, y_train, y_test= train_test_split(
                                                X, y,
                                                test_size=0.2,
                                                random_state=53)



In [15]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
# Calculate precision, recall, f1-score, and support
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Calculate precision, recall, f1-score, and support for each class
metrics = precision_recall_fscore_support(y_test, y_pred)

# Calculate macro and weighted averages
macro_avg = precision_recall_fscore_support(y_test, y_pred, average='macro')
weighted_avg = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Create a DataFrame to store these metrics
df_metrics = pd.DataFrame(list(metrics), index=['Precision', 'Recall', 'F1-score', 'Support']).T

# Additional metrics as a dictionary
additional_rows = [
    {'Class': 'macro avg', 'Precision': macro_avg[0], 'Recall': macro_avg[1], 'F1-score': macro_avg[2], 'Support': ' '},
    {'Class': 'weighted avg', 'Precision': weighted_avg[0], 'Recall': weighted_avg[1], 'F1-score': weighted_avg[2], 'Support': ' '},
    {'Class': 'accuracy', 'Precision': accuracy, 'Recall': ' ', 'F1-score': ' ', 'Support': ' '}
]

# Convert dictionary to DataFrame and concatenate with the existing one
df_additional = pd.DataFrame(additional_rows)
df_metrics_final = pd.concat([df_metrics, df_additional], ignore_index=True)

# Adjust columns for the entire DataFrame
df_metrics_final['Class'] = df_metrics_final.index.map(lambda x: f'Class {x}' if x < len(df_metrics) else df_metrics_final.loc[x, 'Class'])

# Rearrange columns to match the classification_report output
df_metrics_final = df_metrics_final[['Class', 'Precision', 'Recall', 'F1-score', 'Support']]

print(df_metrics_final)

          Class  Precision    Recall  F1-score  Support
0       Class 0   0.848644   0.60582  0.706962   7182.0
1       Class 1   0.834444  0.948421   0.88779  15045.0
2     macro avg   0.841544  0.777121  0.797376         
3  weighted avg   0.839033   0.83772  0.829361         
4      accuracy   0.837720                             


C:\Users\diego\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



param_grid = {'max_depth':range(1, 5),
                'learning_rate':(0.005, 0.05, 0.5)}
 
grid = GridSearchCV(xgb.XGBClassifier(random_state=0), 
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')
    
grid.fit(X_train, y_train)
pred1 = grid.predict(X_test)
print (classification_report(y_test,pred1))

ModuleNotFoundError: No module named 'xgboost'

In [17]:
!pip install xgb

ERROR: Could not find a version that satisfies the requirement xgb (from versions: none)
ERROR: No matching distribution found for xgb


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report
import numpy as np

# Define a simple neural network model for binary classification
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),  # Input layer with ReLU activation
    Dense(64, activation='relu'),  # Hidden layer with ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model with optimizer, loss function, and metrics for binary classification
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model on the training dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions on the testing dataset
pred_probs = model.predict(X_test)
pred_nn = np.round(pred_probs).astype(int)  # Convert probabilities to binary class labels

# Generate a classification report to evaluate the model
df = pd.DataFrame(classification_report(y_test, pred_nn))
df

C:\Users\diego\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8169 - loss: 0.4417
Epoch 2/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8421 - loss: 0.3995
Epoch 3/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8470 - loss: 0.3908
Epoch 4/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8500 - loss: 0.3837
Epoch 5/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8527 - loss: 0.3838
Epoch 6/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8517 - loss: 0.3839
Epoch 7/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8535 - loss: 0.3810
Epoch 8/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8516 - loss: 0.3841
Epoch 9/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8542 - loss: 0.3768
Epoch 10/10
2779/2779 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8531 - loss: 0.3792
695/695 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step
              precision    recall  f1-score   support

           0       0.

In [19]:
# Generate a classification report to evaluate the model
df = pd.DataFrame(classification_report(y_test, pred_nn))
df

ValueError: DataFrame constructor not properly called!

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('clf', LinearSVC(dual=False))  # 'clf' is the name we give to the LinearSVC step
])



# Define a very simple parameter grid, could be expanded based on need
param_grid = {
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['squared_hinge'],
    'clf__tol': [1e-4, 1e-3, 1e-2],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__max_iter': [1000, 2000, 3000],
    'clf__intercept_scaling': [1, 10, 100]
}


# Initialize GridSearchCV with the SVC estimator and the simple param grid
grid = RandomizedSearchCV(pipeline, param_grid, verbose=3, cv=10)

# Train the model on the training dataset
grid.fit(X_train, y_train)

# Make predictions on the testing dataset using the best model found
pred_svm = grid.predict(X_test)

# Generate a classification report to evaluate the model
print(classification_report(y_test, pred_svm))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.0001;, score=0.752 total time=  12.0s
[CV 2/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.0001;, score=0.752 total time=  11.8s
[CV 3/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.0001;, score=0.752 total time=   9.9s
[CV 4/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.0001;, score=0.755 total time=  10.7s
[CV 5/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, clf__tol=0.0001;, score=0.752 total time=  11.8s
[CV 6/10] END clf__C=0.1, clf__intercept_scaling=10, clf__loss=squared_hinge, clf__max_iter=2000, clf__penalty=l1, cl

[CV 1/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.751 total time=   0.4s
[CV 2/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.753 total time=   0.4s
[CV 3/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.752 total time=   0.4s
[CV 4/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.754 total time=   0.4s
[CV 5/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.753 total time=   0.4s
[CV 6/10] END clf__C=0.01, clf__intercept_scaling=1, clf__loss=squared_hinge, clf__max_iter=3000, clf__penalty=l2, clf__tol=0.01;, score=0.751 total time=   0.4s
[CV 7/10] END clf__C=0.01, c

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Adjusting the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # Number of trees in the forest
    'max_depth': range(1, 5)  # Maximum depth of the tree
}

grid = GridSearchCV(RandomForestClassifier(random_state=0),
                    param_grid=param_grid, cv=10, verbose=1, scoring='accuracy')

# Fit to the training data
grid.fit(X_train, y_train)

# Predict on the test data
pred1 = grid.predict(X_test)

# Print the classification report
print(classification_report(y_test, pred1))

Fitting 10 folds for each of 16 candidates, totalling 160 fits
              precision    recall  f1-score   support

           0       0.68      0.43      0.52     59348
           1       0.77      0.90      0.83    124719

    accuracy                           0.75    184067
   macro avg       0.72      0.66      0.68    184067
weighted avg       0.74      0.75      0.73    184067

